In [ ]:
import numpy as np
import pandas as pd

In [ ]:
bus_groups_replaceinfo = pd.read_csv("../data/bus_groups_replaceinfo.csv")

bus_groups_replaceinfo = bus_groups_replaceinfo[
    ["bus_name", "group", "purchase_date", "1st_rep_date"]
]
bus_groups_replaceinfo["purchase_date"] = pd.to_datetime(
    bus_groups_replaceinfo["purchase_date"], format="%Y-%m"
)

bus_groups_replaceinfo

In [ ]:
def calculate_months_diff(date1: pd.Series, date2: pd.Series) -> pd.Series:
    result = np.where(
        date1.isna() | date2.isna(),
        np.nan,
        (date2.dt.year - date1.dt.year) * 12 + (date2.dt.month - date1.dt.month) + 1,
    )
    return pd.Series(result)

In [ ]:
bus_groups_no_replace = bus_groups_replaceinfo[
    bus_groups_replaceinfo["1st_rep_date"].isna()
].copy()
bus_groups_no_replace.reset_index(drop=True, inplace=True)

bus_groups_no_replace["end_date"] = pd.to_datetime("1985-05", format="%Y-%m")
bus_groups_no_replace["flapsed_time"] = calculate_months_diff(
    bus_groups_no_replace["purchase_date"], bus_groups_no_replace["end_date"]
)
bus_groups_no_replace.drop(
    columns=["1st_rep_date", "purchase_date", "end_date"], inplace=True
)
bus_groups_no_replace

In [ ]:
bus_groups_no_replace.value_counts("group")

In [ ]:
bus_groups_time_series = pd.read_csv("../data/bus_groups_timeseries.csv", index_col=0)
bus_groups_time_series

In [ ]:
no_replace_bus_names = bus_groups_no_replace["bus_name"].astype(str).tolist()

miles_1985 = bus_groups_time_series[no_replace_bus_names].loc["1985-05"]
miles_1985 = miles_1985.reset_index(name="miles_1985").rename(
    columns={"index": "bus_name"}
)
miles_1985["bus_name"] = miles_1985["bus_name"].astype(int)

bus_groups_no_replace = pd.merge(bus_groups_no_replace, miles_1985, on="bus_name")
bus_groups_no_replace

In [ ]:
bus_groups_no_replace[["group", "miles_1985", "flapsed_time"]].groupby("group").agg(
    ["max", "min", "mean", "std"]
).round(2)